In [40]:
import TFO_dataset
from math import pi
from sklearn.gaussian_process import *
from inverse_modelling_tfo.data.intensity_interpolation import interpolate_exp_chunk, get_interpolate_fit_params
from inverse_modelling_tfo.data import normalize_zero_mean 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from typing import Union, List

In [41]:
# train_data = pd.read_pickle(r'/home/rraiyan/personal_projects/tfo_inverse_modelling/data/intensity/intensity_averaged_sim_data.pkl')
train_data = pd.read_pickle(
    r'/home/rraiyan/personal_projects/tfo_inverse_modelling/data/intensity/intensity_summed_sim_data_equidistance_detector2.pkl')
# Only for intensity_summed_sim_data_equidistance_detector.pkl
sdd = train_data['SDD'].to_numpy()[:20]
detector_count = [11, 16, 22, 27, 32, 38, 43, 48, 53,
                  59, 64, 69, 75, 80, 85, 90, 96, 101, 106, 111]
sdd_to_detector_count_map = {
    dist: count for dist, count in zip(sdd, detector_count)}
train_data['Intensity'] /= train_data['SDD'].map(
    sdd_to_detector_count_map)

# For the other cases
# train_data['Intensity'] /= 20   # Normalize by the number of detectors per ring

train_data['Intensity'] /= 1e9  # Photon count/Initial intensity



interpolated_training_data = get_interpolate_fit_params(
    train_data, weights=[1, -2])

interpolated_training_data.head()


,Wave Int,Uterus Thickness,Maternal Wall Thickness,Maternal Mu_a,Fetal Mu_a,alpha0,alpha1,alpha2,alpha3
0,2.0,5.0,26.0,0.005,0.05,-16.959114,1.036984,-29.212411,42.008351
1,2.0,5.0,26.0,0.006,0.05,-17.818238,1.068003,-30.353526,43.906572
2,2.0,5.0,26.0,0.007,0.05,-18.635541,1.097803,-31.440929,45.713937
3,2.0,5.0,26.0,0.008,0.05,-19.415485,1.126506,-32.480971,47.441001
4,2.0,5.0,26.0,0.009,0.05,-20.161950,1.154220,-33.478903,49.096596


In [42]:

# Incorporate both wavelengths by moving to a Wide Format from Long Format
interpolated_training_data = interpolated_training_data.pivot_table(
    index=['Uterus Thickness', 'Maternal Wall Thickness', 'Maternal Mu_a', 'Fetal Mu_a'], columns='Wave Int', values=['alpha0', 'alpha1', 'alpha2', 'alpha3']).reset_index()


print(interpolated_training_data.columns)

def _renaming_func(x, y):
    if y == '':
        return f'{x}'
    else:
        return f'{x}_{int(y)}'


interpolated_training_data.columns = [_renaming_func(
    x, y) for x, y in interpolated_training_data.columns]
interpolated_training_data.head()


MultiIndex([(       'Uterus Thickness',  ''),
            ('Maternal Wall Thickness',  ''),
            (          'Maternal Mu_a',  ''),
            (             'Fetal Mu_a',  ''),
            (                 'alpha0', 1.0),
            (                 'alpha0', 2.0),
            (                 'alpha1', 1.0),
            (                 'alpha1', 2.0),
            (                 'alpha2', 1.0),
            (                 'alpha2', 2.0),
            (                 'alpha3', 1.0),
            (                 'alpha3', 2.0)],
           names=[None, 'Wave Int'])


,Uterus Thickness,Maternal Wall Thickness,Maternal Mu_a,Fetal Mu_a,alpha0_1,alpha0_2,alpha1_1,alpha1_2,alpha2_1,alpha2_2,alpha3_1,alpha3_2
0,5.0,2.0,0.005,0.05,-18.731813,-20.444467,1.136306,1.165762,-33.474455,-38.806152,48.387194,56.459243
1,5.0,2.0,0.005,0.06,-19.395468,-22.200493,1.137524,1.206672,-33.947994,-40.409166,49.381108,59.436939
2,5.0,2.0,0.005,0.07,-19.993693,-23.640858,1.140056,1.240522,-34.389651,-41.728256,50.292328,61.884206
3,5.0,2.0,0.005,0.08,-20.539531,-24.856917,1.143432,1.269353,-34.803629,-42.845508,51.135119,63.954438
4,5.0,2.0,0.005,0.09,-21.041806,-25.905711,1.147346,1.294433,-35.192921,-43.812067,51.919289,65.743294


In [43]:
# Fitting only on both WV

# Create Features & Normalize the fitting params
interpolated_training_data['Bias Ratio'] = interpolated_training_data['alpha0_1'] / interpolated_training_data['alpha0_2'] 

X = interpolated_training_data[['Bias Ratio', 'alpha1_1', 'alpha1_2', 'alpha2_1', 'alpha2_2', 'alpha3_1', 'alpha3_2']].to_numpy()
alpha_scaler = preprocessing.StandardScaler().fit(X)
X = alpha_scaler.transform(X)

y = interpolated_training_data[['Fetal Mu_a']].to_numpy().flatten()
# y = interpolated_training_data[['Maternal Mu_a']].to_numpy().flatten()
y_scaler = preprocessing.StandardScaler().fit(y.reshape(-1, 1))
y = y_scaler.transform(y.reshape(-1, 1)).flatten()

rng = np.random.RandomState(1)
random_indices = rng.choice(np.arange(y.size), size=y.size, replace=False)
training_count = int(y.size * 1)  # 80% Training Data
training_indices = random_indices[:training_count]
test_indices = random_indices[training_count:]

X_train, y_train = X[training_indices], y[training_indices]
X_test, y_test = X[test_indices], y[test_indices]

In [44]:
# kernel = 1 * kernels.RBF(length_scale=1.0, length_scale_bounds=(1e-4, 1e1))
kernel = 1 * kernels.Matern(length_scale=1.0, length_scale_bounds=(1e-6, 1e-2))
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
gp.fit(X_train, y_train)
gp.kernel_

/home/rraiyan/cybercat/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


1.03**2 * Matern(length_scale=5.21e-05, nu=1.5)

In [45]:
# # PRedict on Simulation
# X_test = X_train
# y_test = y_train
# mean_prediction, std_prediction = gp.predict(X_test, return_std=True)
# mae = np.abs(mean_prediction - y_test)
# mse = np.square(mean_prediction - y_test)
# df = pd.DataFrame(
#     {
#         'True a0' : X_test[:, 0],
#         'True a1' : X_test[:, 1],
#         'True a2' : X_test[:, 2],
#         'True a3' : X_test[:, 3],
#         'True y'  : y_test,
#         'Prediction' : mean_prediction,
#         'Confidence' : std_prediction,
#         'MAE(%)' : mae * 100,
#         'MSE(%)' : mse * 100,
#     }
# )
# pd.set_option('display.max_rows', 1200)
# df

In [46]:
# df['MAE(%)'].describe()

In [47]:
def prepare_patient_ppg(ppg_data : pd.DataFrame, sample_number : Union[int, List], SDD = [15, 30, 45, 70, 100]) -> np.ndarray:
    """Prepare PPG data to be used directly into the GPR prediction.

    Args:
        ppg_data (pd.DataFrame): PPG data Dataframe. You can feed data directly from the the TFO_dataset package.
        (Note: This should ideally be the optically normalized data)
        sample_number (int): which sample to choose. You can either pass a single integer or an array
        SDD (_type_, optional): Detector distances in TFO device(in mm). Defaults to SDD=[15, 30, 45, 70, 100].
    """
    # The code is generalized to run on any array. make necessary conversions 
    if isinstance(sample_number, int):
        sample_number = [sample_number]
    
    patient_features = []
    for sample_point in sample_number:
        # Pick a point in time
        spatial_intensity = ppg_data.iloc[sample_point].copy()  # at 300s with 80Hz sampling freq.
        spatial_intensity *=  pi * 4   # from unit area -> pi r^2 area -> match simulation
        # Reshape ppg data to fit the format
        spatial_intensity_wv1 = pd.DataFrame(data={
            'SDD' : SDD,
            'Intensity' : spatial_intensity.to_numpy()[:5]
        })
        spatial_intensity_wv2 = pd.DataFrame(data={
            'SDD' : SDD,
            'Intensity' : spatial_intensity.to_numpy()[5:]
        })
        alpha_wv1 = interpolate_exp_chunk(spatial_intensity_wv1, weights=[1.0, -2.0], return_alpha=True).flatten()
        alpha_wv2 = interpolate_exp_chunk(spatial_intensity_wv2, weights=[1.0, -2.0], return_alpha=True).flatten()
        patient_features.append([alpha_wv1[0]/alpha_wv2[0], alpha_wv1[1], alpha_wv2[1], alpha_wv1[2], alpha_wv2[2], alpha_wv1[3], alpha_wv2[3]])
        
    return np.array(patient_features)
    

In [48]:
# Predict on reallife data
sheep_id = 23
data = TFO_dataset.SheepData('iq_demod_optical').get(sheep_id)
print(TFO_dataset.SheepData('iq_demod_optical').get_tuple(sheep_id))

features = prepare_patient_ppg(data, [1000, 2000, 3000])

# Create a DF for better viz.
print('Non- normalized Features')
feature_names = [f'f{i + 1}' for i in range(7)]
feature_df = pd.DataFrame(columns=feature_names, data=features)
print(feature_df)
features = alpha_scaler.transform(features)

(2, 1, 'sp2021', 'Recovery')
Non- normalized Features
         f1        f2        f3          f4          f5          f6  \
0  0.923122 -7.762932 -8.427821  432.893302  469.738907 -918.699976   
1  0.952615 -8.271182 -8.689905  460.316041  483.668010 -976.018981   
2  0.913762 -7.339342 -8.054755  409.465326  449.041880 -869.461025   

            f7  
0  -996.130024  
1 -1025.140742  
2  -952.596061  


In [49]:
estimate, confidence = gp.predict(features, return_cov=True)
# estimate, confidence = gp.predict(X_train[0, :].reshape(1, -1), return_std=True)
print(y_scaler.inverse_transform(np.array(estimate).reshape(-1, 1)))
print(confidence)

[[0.07]
 [0.07]
 [0.07]]
[[1.06042771 0.         0.        ]
 [0.         1.06042771 0.        ]
 [0.         0.         1.06042771]]


In [50]:
print(f'alpha means (From training) : {alpha_scaler.mean_}')
print(f'alpha variance (From training) : {alpha_scaler.var_}')

alpha means (From training) : [  0.94525161   1.16190502   1.16155528 -33.70954645 -35.10835354
  49.53349681  52.43872191]
alpha variance (From training) : [1.35868092e-02 2.25335223e-03 1.01410463e-02 4.60485405e+00
 3.14067449e+01 1.46363908e+01 1.03779584e+02]
